<a href="https://colab.research.google.com/github/rinogrego/Learning-LLM/blob/main/explorations/Fine-Tuning-Mistral-7B-Skripsi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Fine-Tuning Mistral 7b in Google Colab with QLoRA

Referensi:
- https://medium.com/@codersama/fine-tuning-mistral-7b-in-google-colab-with-qlora-complete-guide-60e12d437cca

These are the steps:
- convert the code base to a synthetic dialogue-based train-test dataset
  - **changed to own data**
- fine-tune with QLoRA
- evaluate the new model
- evaluate the base model + GPT-4
- (optional) merge the adapter with the base model
- (optional) quantize the model and get GGUF format

In [ ]:
import os

os.environ["OPENAI_API_KEY"] = ""
os.environ["HUGGINGFACEHUB_API_TOKEN"] = ""
os.environ["GOOGLE_API_KEY"] = ""

In [ ]:
!pip install langchain
!pip install -q -U google-generativeai
%pip install --upgrade --quiet  langchain-google-genai
!pip install pypdf

from google.colab import output
output.clear()

## Generate Q/A Pairs

In [ ]:
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains import RetrievalQA
from langchain.vectorstores import FAISS
from langchain.agents import initialize_agent, AgentType

from langchain_google_genai import GoogleGenerativeAI

In [ ]:
llm = GoogleGenerativeAI(model="gemini-pro", google_api_key=os.environ["GOOGLE_API_KEY"])
print(
    llm.invoke(
        "Jelaskan tentang congestive heart failure"
    )
)

**Gagal Jantung Kongestif (Congestive Heart Failure)**

Gagal jantung kongestif (CHF) adalah suatu kondisi di mana jantung tidak dapat memompa cukup darah untuk memenuhi kebutuhan tubuh. Hal ini menyebabkan penumpukan cairan di paru-paru, hati, dan anggota tubuh lainnya.

**Penyebab**

CHF dapat disebabkan oleh berbagai kondisi yang melemahkan otot jantung atau menghalangi aliran darah, antara lain:

* Penyakit arteri koroner
* Hipertensi (tekanan darah tinggi)
* Penyakit katup jantung
* Kardiomiopati (penyakit otot jantung)
* Diabetes
* Obesitas
* Riwayat keluarga CHF

**Gejala**

Gejala CHF dapat bervariasi tergantung pada tingkat keparahannya. Gejala umum meliputi:

* Sesak napas, terutama saat berbaring atau berolahraga
* Kelelahan
* Pembengkakan pada kaki, pergelangan kaki, dan perut
* Penambahan berat badan yang cepat
* Batuk terus-menerus
* Mual dan muntah
* Pusing atau pingsan

**Diagnosis**

Untuk mendiagnosis CHF, dokter akan melakukan pemeriksaan fisik, meninjau riwayat kese

In [ ]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("/content/Skripsi-Bab 2.pdf")
loader

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100)
pages = loader.load_and_split(text_splitter=text_splitter)
# pages = loader.load_and_split()

In [ ]:
for idx, page in enumerate(pages):
    print("CHUNK PAGE    :", idx + 1)
    print("DOCUMENT PAGE :", page.metadata['page'] + 1)
    for key, value in page:
        if key == "page_content":
            print("{}:\n{}\n".format(key, value))
        else:
            print("{}: {}\n".format(key, value))
    print("="*50)

CHUNK PAGE    : 1
DOCUMENT PAGE : 1
page_content:
1 
Universitas Indonesia  BAB 2  
LANDASAN TEORI  
 
Pada subbab ini akan dijelaskan berbagai teori yang menjadi landasan penelitian dalam 
skripsi ini.  
2.1. Inde ks Heart Rate Variability  
Indeks heart rate variability (HRV) merupakan metrik pengukur HRV yang diperoleh 
melalui informasi interval RR yang didapatkan dari suatu rekaman  detak jantung . Pada 
umumnya, nilai indeks HRV dapat diperoleh melalui rekaman EKG dengan panjang 
berapapun. Istilah long-term HRV dan short -term HRV adalah perhitungan indeks HRV 
berdasarkan panjang masing -masing rekaman.  
 Untuk long-term HRV sendiri terdapat variasi panjang rekaman detak jantung 
yang dijadikan bahan perhitungan. Pada u mumnya , long-term HRV berarti perhitungan 
indeks HRV menggunakan rekaman EKG yang panjang waktunya selama 24 jam (Jovic, 
2019). Namun, ada juga variasi perhitungan yang tetap dapat dikategorikan long-term 
HRV. Binici dkk (2011) menggunakan rekaman sepanjang

In [ ]:
SYSTEM_PROMPT = """# you take a document of undergraduate thesis and create 10 question/answer pairs for it.
# you are GREAT at extracting the most important information out of these thesis texts.
# questions should have context.
# DO NOT say 'the code' or 'this code' in your questions and do not refer to it at all.
# mention the filename in all questions.
# your answer MUST be a valid json format, a list of 10 objects, each object has 4 text fields: filename, topic, Q(question), A(answer)
# any '\n' in the text fields MUST be '\\n' so that when reading it later on, we won't run into any issues

# example output with 2 question/answer
[
  {
    "filename": "Analisis Kinerja Model XGBoost dalam Mendeteksi Congestive Heart Failure pada Short-term Heart Rate Variability",
    "topic": "congestive heart failure",
    "Q": "Apa itu Congestive Heart Failure? Apa saja jenis-jenis gejala yang mungkin?",
    "A": "Congestive Heart Failure (CHF) adalah kondisi dimana jantung tidak dapat memompa darah yang cukup ke seluruh bagian tubuh. Hal ini mengakibatkan beberapa bagian tubuh tidak mendapatkan suplai darah yang cukup. Beberapa gejala yang mungkin terjadi pada penderita CHF adalah sesak nafas, nyeri dada, palpitasi atau jantung berdebar-debar, anoreksia atau permasalahan mental terhadap makanan, dan kelelahan."
  },
  {
    "filename": "Analisis Kinerja Model XGBoost dalam Mendeteksi Congestive Heart Failure pada Short-term Heart Rate Variability",
    "topic": "congestive heart failure",
    "Q": "Jelaskan tentang kelas keparahan CHF",
    "A": "Kelas keparahan CHF merujuk kepada tingkat keparahan dari CHF. New York Heart Association (NYHA) membagi tingkat keparahan CHF yang disebut kelas fungsional CHF menjadi empat berdasarkan waktu munculnya gejala yang diderita oleh pasien, yaitu NYHA Class I untuk pasien yang mengalami gejala ketika beraktivitas berlebih dari yang biasanya, NYHA Class II untuk pasien yang mengalami gejala ketika beraktivitas normal, NYHA Class III untuk pasien yang mengalami gejala ketika beraktivitas minimal, dan NYHA Class IV untuk pasien yang mengalami gejala ketika beristirahat."
  }
]
# end of examples.

# this is the chunk of the text from the thesis:
"""

In [ ]:
llm = GoogleGenerativeAI(model="gemini-pro", google_api_key=os.environ["GOOGLE_API_KEY"])

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100)
pages = loader.load_and_split(text_splitter=text_splitter)
pages[0].page_content # approx. 57 words for chunk_size 500

'1 \nUniversitas Indonesia  BAB 2  \nLANDASAN TEORI  \n \nPada subbab ini akan dijelaskan berbagai teori yang menjadi landasan penelitian dalam \nskripsi ini.  \n2.1. Inde ks Heart Rate Variability  \nIndeks heart rate variability (HRV) merupakan metrik pengukur HRV yang diperoleh \nmelalui informasi interval RR yang didapatkan dari suatu rekaman  detak jantung . Pada \numumnya, nilai indeks HRV dapat diperoleh melalui rekaman EKG dengan panjang'

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
pages = loader.load_and_split(text_splitter=text_splitter)
pages[0].page_content # approx. 126 words for chunk_size 1000

'1 \nUniversitas Indonesia  BAB 2  \nLANDASAN TEORI  \n \nPada subbab ini akan dijelaskan berbagai teori yang menjadi landasan penelitian dalam \nskripsi ini.  \n2.1. Inde ks Heart Rate Variability  \nIndeks heart rate variability (HRV) merupakan metrik pengukur HRV yang diperoleh \nmelalui informasi interval RR yang didapatkan dari suatu rekaman  detak jantung . Pada \numumnya, nilai indeks HRV dapat diperoleh melalui rekaman EKG dengan panjang \nberapapun. Istilah long-term HRV dan short -term HRV adalah perhitungan indeks HRV \nberdasarkan panjang masing -masing rekaman.  \n Untuk long-term HRV sendiri terdapat variasi panjang rekaman detak jantung \nyang dijadikan bahan perhitungan. Pada u mumnya , long-term HRV berarti perhitungan \nindeks HRV menggunakan rekaman EKG yang panjang waktunya selama 24 jam (Jovic, \n2019). Namun, ada juga variasi perhitungan yang tetap dapat dikategorikan long-term \nHRV. Binici dkk (2011) menggunakan rekaman sepanjang 48 jam untuk memperoleh'

In [ ]:
len(pages)

In [ ]:
page_content = pages[0].page_content

GENERATE_QA_PROMPT = f"{SYSTEM_PROMPT}\n{page_content}"
LLM_OUTPUT = llm.invoke(GENERATE_QA_PROMPT)
print(LLM_OUTPUT)

[
  {
    "filename": "LANDASAN TEORI",
    "topic": "heart rate variability",
    "Q": "Apa itu indeks Heart Rate Variability (HRV)?",
    "A": "Indeks HRV adalah metrik yang mengukur variabilitas detak jantung yang diperoleh dari interval RR dalam rekaman detak jantung."
  },
  {
    "filename": "LANDASAN TEORI",
    "topic": "heart rate variability",
    "Q": "Apa perbedaan antara long-term HRV dan short-term HRV?",
    "A": "Perbedaannya terletak pada panjang rekaman detak jantung yang digunakan dalam perhitungan indeks HRV. Long-term HRV umumnya menggunakan rekaman selama 24 jam atau lebih, sedangkan short-term HRV menggunakan rekaman yang lebih pendek."
  },
  {
    "filename": "LANDASAN TEORI",
    "topic": "heart rate variability",
    "Q": "Berapa lama rekaman EKG yang biasanya digunakan untuk menghitung long-term HRV?",
    "A": "Umumnya, rekaman EKG selama 24 jam digunakan untuk menghitung long-term HRV."
  },
  {
    "filename": "LANDASAN TEORI",
    "topic": "heart rate va

In [ ]:
page_content = pages[1].page_content

GENERATE_QA_PROMPT = f"{SYSTEM_PROMPT}\n{page_content}"
LLM_OUTPUT = llm.invoke(GENERATE_QA_PROMPT)
print(LLM_OUTPUT)

[
  {
    "filename": "Analisis Kinerja Model XGBoost dalam Mendeteksi Congestive Heart Failure pada Short-term Heart Rate Variability",
    "topic": "long-term HRV",
    "Q": "Bagaimana cara menghitung long-term HRV?",
    "A": "Long-term HRV dihitung menggunakan rekaman detak jantung selama 24 jam atau lebih."
  },
  {
    "filename": "Analisis Kinerja Model XGBoost dalam Mendeteksi Congestive Heart Failure pada Short-term Heart Rate Variability",
    "topic": "long-term HRV",
    "Q": "Apa saja penelitian yang menggunakan long-term HRV?",
    "A": "Binici dkk (2011) menggunakan rekaman sepanjang 48 jam untuk memperoleh long-term HRV dan menganalisis keterkaitan penyakit stroke dengan nilai HRV pada waktu malam.\nAlabdulgader dkk (2018) menggunakan rekaman sepanjang 72 jam untuk menghitung long-term HRV dan menganalisis pengaruh aktivitas geomagnetik lingkungan dan solar."
  },
  {
    "filename": "Analisis Kinerja Model XGBoost dalam Mendeteksi Congestive Heart Failure pada Short-te

In [ ]:
import datetime

now = datetime.datetime.now()
page_content = pages[19].page_content

GENERATE_QA_PROMPT = f"{SYSTEM_PROMPT}\n{page_content}"
LLM_OUTPUT = llm.invoke(GENERATE_QA_PROMPT)
print(LLM_OUTPUT)
print("="*100)
print("Time to generate QA:", datetime.datetime.now() - now)
print("="*100)

[
  {
    "filename": "Penerapan Ensemble Learning untuk Mendeteksi Penyakit Jantung Koroner pada Pemeriksaan Elektrokardiogram",
    "topic": "decision tree",
    "Q": "Jelaskan tentang visualisasi hasil keputusan decision tree",
    "A": "Visualisasi hasil keputusan decision tree dapat digambarkan melalui simpul-simpul keputusan yang membentuk sebuah pohon. Setiap simpul keputusan mewakili sebuah atribut atau fitur dari data, dan setiap cabang yang keluar dari simpul mewakili nilai yang mungkin dari atribut tersebut. Daun-daun dari pohon mewakili keputusan akhir atau prediksi yang dibuat oleh model."
  },
  {
    "filename": "Penerapan Ensemble Learning untuk Mendeteksi Penyakit Jantung Koroner pada Pemeriksaan Elektrokardiogram",
    "topic": "gradient boosting machine",
    "Q": "Apa itu Gradient Boosting Machine (GBM)?",
    "A": "Gradient Boosting Machine (GBM) adalah model machine learning yang memanfaatkan teknik ensemble learning dalam konstruksi modelnya. Teknik ensemble lear

In [ ]:
type(json.loads(LLM_OUTPUT))

list

In [ ]:
import pandas as pd
import json

data = json.loads(LLM_OUTPUT)
df = pd.DataFrame(data)
df

,filename,topic,Q,A
0,Penerapan Ensemble Learning untuk Mendeteksi P...,decision tree,Jelaskan tentang visualisasi hasil keputusan d...,Visualisasi hasil keputusan decision tree dapa...
1,Penerapan Ensemble Learning untuk Mendeteksi P...,gradient boosting machine,Apa itu Gradient Boosting Machine (GBM)?,Gradient Boosting Machine (GBM) adalah model m...
2,Penerapan Ensemble Learning untuk Mendeteksi P...,gradient boosting machine,Bagaimana cara kerja GBM dalam proses training?,"Dalam proses training, GBM memanfaatkan konsep..."
3,Penerapan Ensemble Learning untuk Mendeteksi P...,gradient boosting machine,Apa fungsi dari weak learner dalam GBM?,Weak learner dalam GBM adalah model-model Deci...
4,Penerapan Ensemble Learning untuk Mendeteksi P...,gradient boosting machine,Jelaskan tentang konsep optimasi yang digunaka...,Konsep optimasi yang digunakan dalam GBM adala...
5,Penerapan Ensemble Learning untuk Mendeteksi P...,gradient boosting machine,Apa tujuan dari optimisasi dalam GBM?,Tujuan dari optimisasi dalam GBM adalah untuk ...
6,Penerapan Ensemble Learning untuk Mendeteksi P...,gradient boosting machine,Apa fungsi dari fungsi loss dalam GBM?,Fungsi loss dalam GBM berfungsi sebagai target...
7,Penerapan Ensemble Learning untuk Mendeteksi P...,decision tree,Bagaimana visualisasi hasil keputusan decision...,Visualisasi hasil keputusan decision tree memb...
8,Penerapan Ensemble Learning untuk Mendeteksi P...,gradient boosting machine,Apa perbedaan antara GBM dan ensemble learning...,Tidak ada informasi tentang perbedaan antara G...
9,Penerapan Ensemble Learning untuk Mendeteksi P...,gradient boosting machine,Bagaimana GBM digunakan dalam konteks medis?,Tidak ada informasi tentang penggunaan GBM dal...


In [ ]:
GoogleGenerativeAI

langchain_google_genai.llms.GoogleGenerativeAI

### Define the System Prompt and Function

In [ ]:
def generate_system_prompt(qa_num=5):
    # https://stackoverflow.com/questions/3877623/can-you-have-variables-within-triple-quotes-if-so-how
    # HOW TO PUT qa_num VARIABLE INSIDE TRIPLE QUOTE STRING ???
    SYSTEM_PROMPT = """# you take a document of undergraduate thesis and create 10 question/answer pairs for it.
    # you are GREAT at extracting the most important information out of these thesis texts.
    # questions should have context.
    # only give answers and questions in Bahasa Indonesia.
    # mention the filename in all questions.
    # your answer MUST be a valid json format, a list of 10 objects, each object has 3 text fields: topic, Q(question), A(answer)
    # any '\n' in the text fields MUST be '\\n' so that when reading it later on, we won't run into any issues

    # example output with 3 question/answer
    [
    {
        "topic": "congestive heart failure",
        "Q": "Apa itu Congestive Heart Failure? Apa saja jenis-jenis gejala yang mungkin?",
        "A": "Congestive Heart Failure (CHF) adalah kondisi dimana jantung tidak dapat memompa darah yang cukup ke seluruh bagian tubuh. Hal ini mengakibatkan beberapa bagian tubuh tidak mendapatkan suplai darah yang cukup. Beberapa gejala yang mungkin terjadi pada penderita CHF adalah sesak nafas, nyeri dada, palpitasi atau jantung berdebar-debar, anoreksia atau permasalahan mental terhadap makanan, dan kelelahan."
    },
    {
        "topic": "congestive heart failure",
        "Q": "Jelaskan tentang kelas keparahan CHF",
        "A": "Kelas keparahan CHF merujuk kepada tingkat keparahan dari CHF. New York Heart Association (NYHA) membagi tingkat keparahan CHF yang disebut kelas fungsional CHF menjadi empat berdasarkan waktu munculnya gejala yang diderita oleh pasien, yaitu NYHA Class I untuk pasien yang mengalami gejala ketika beraktivitas berlebih dari yang biasanya, NYHA Class II untuk pasien yang mengalami gejala ketika beraktivitas normal, NYHA Class III untuk pasien yang mengalami gejala ketika beraktivitas minimal, dan NYHA Class IV untuk pasien yang mengalami gejala ketika beristirahat."
    },
    {
        "topic": "congestive heart failure",
        "Q": "Apa kepanjangan dari XGBoost?",
        "A": "eXtreme Gradient Boosting"
    }
    ]
    # end of examples.

    # this is the chunk of the text from the thesis:
    """
    return SYSTEM_PROMPT

In [ ]:
from langchain_community.document_loaders import PyPDFLoader
import pandas as pd
import json
import datetime
import time

def generate_qa(
    filepath = None,
    llm = None,
    # qa_num = 5,
) -> pd.DataFrame:
    start_time = datetime.datetime.now()
    loader = PyPDFLoader(filepath)
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
    pages = loader.load_and_split(text_splitter=text_splitter)

    llm = GoogleGenerativeAI(model="gemini-pro", google_api_key=os.environ["GOOGLE_API_KEY"])

    SYSTEM_PROMPT = generate_system_prompt()
    lists_of_qa = []
    for idx, page in enumerate(pages):
        page_start_time = datetime.datetime.now()
        print("CHUNK PAGE    :", idx + 1)
        print("DOCUMENT PAGE :", page.metadata['page'] + 1)
        page_content = page.page_content

        GENERATE_QA_PROMPT = f"{SYSTEM_PROMPT}\n{page_content}"
        LLM_OUTPUT = llm.invoke(GENERATE_QA_PROMPT) # outputs 'string'
        try:
            data = json.loads(LLM_OUTPUT) # outputs 'list'

            lists_of_qa += data # list addition
        except ValueError: # include JSONDecodeError
            print(" Error 'JSONDecodeError' at chunk {}, document page {}".format(idx + 1, page.metadata['page']+1))
            print(" Skipping...")

        page_end_time = datetime.datetime.now()
        print("TIME TAKEN    :", page_end_time - page_start_time)

        time.sleep(1)
        print("="*50)
        # if idx == 5:
        #     break

    df = pd.DataFrame(lists_of_qa)
    # df = df.map(lambda x: x.replace('\\n', '\n')) # for code-only dataset? idk...
    end_time = datetime.datetime.now()
    print("="*100)
    print("Time taken       :", end_time - start_time)
    print("Num. of Chunks   :", idx+1)
    print("="*100)
    return df

In [ ]:
# for easier dataframe QnA read
pd.set_option('display.max_colwidth', None)

### Generate Q/A Pairs for Bab 2

In [ ]:
df_qa = generate_qa("/content/Skripsi-Bab 2.pdf")

CHUNK PAGE    : 1
DOCUMENT PAGE : 1
TIME TAKEN    : 0:00:14.510609
CHUNK PAGE    : 2
DOCUMENT PAGE : 1
TIME TAKEN    : 0:00:17.657658
CHUNK PAGE    : 3
DOCUMENT PAGE : 1
TIME TAKEN    : 0:00:06.728078
CHUNK PAGE    : 4
DOCUMENT PAGE : 2
TIME TAKEN    : 0:00:09.121165
CHUNK PAGE    : 5
DOCUMENT PAGE : 2
TIME TAKEN    : 0:00:09.447252
CHUNK PAGE    : 6
DOCUMENT PAGE : 3
TIME TAKEN    : 0:00:09.529809
CHUNK PAGE    : 7
DOCUMENT PAGE : 3
TIME TAKEN    : 0:00:11.044065
CHUNK PAGE    : 8
DOCUMENT PAGE : 4
TIME TAKEN    : 0:00:10.053148
CHUNK PAGE    : 9
DOCUMENT PAGE : 4
TIME TAKEN    : 0:00:17.001899
CHUNK PAGE    : 10
DOCUMENT PAGE : 5
TIME TAKEN    : 0:00:11.718184
CHUNK PAGE    : 11
DOCUMENT PAGE : 5
TIME TAKEN    : 0:00:13.011680
CHUNK PAGE    : 12
DOCUMENT PAGE : 6
TIME TAKEN    : 0:00:09.341719
CHUNK PAGE    : 13
DOCUMENT PAGE : 6
TIME TAKEN    : 0:00:13.423455
CHUNK PAGE    : 14
DOCUMENT PAGE : 6
TIME TAKEN    : 0:00:14.366634
CHUNK PAGE    : 15
DOCUMENT PAGE : 7
TIME TAKEN    : 0:00

In [ ]:
df_qa

,topic,Q,A
0,heart rate variability,Apa singkatan dari HRV?,Heart Rate Variability
1,heart rate variability,Bagaimana cara memperoleh nilai indeks HRV?,Melalui rekaman EKG dengan panjang berapapun
2,heart rate variability,Apa perbedaan long-term HRV dan short-term HRV?,Perbedaannya terletak pada panjang rekaman detak jantung yang digunakan untuk perhitungan indeks HRV
3,long-term HRV,Berapa lama waktu rekaman EKG yang umumnya digunakan untuk menghitung long-term HRV?,24 jam
4,long-term HRV,Apakah ada variasi perhitungan long-term HRV selain rekaman 24 jam?,"Ya, ada"
...,...,...,...
213,confusion matrix,Apa nilai False Negative (FN) dalam confusion matrix?,Nilai False Negative (FN) dalam confusion matrix adalah jumlah prediksi negatif yang salah.
214,confusion matrix,Apa nilai True Positive (TP) dalam confusion matrix?,Nilai True Positive (TP) dalam confusion matrix adalah jumlah prediksi positif yang benar.
215,confusion matrix,Apa rumus untuk menghitung akurasi?,Rumus untuk menghitung akurasi adalah: Akurasi = (TP + TN) / (TP + TN + FP + FN)
216,AUC-ROC,Apa rumus untuk menghitung TPR dan FPR?,TPR = Sensitivitas\nFPR = 1 - Sensitivitas


In [ ]:
df_qa.to_excel("df_qa_bab_2_qa-10.xlsx")
df_qa.to_csv("df_qa_bab_2_qa-10.csv")

In [ ]:
pd.read_csv("df_qa_bab_2_qa-10.csv", index_col=[0])

,topic,Q,A
0,heart rate variability,Apa singkatan dari HRV?,Heart Rate Variability
1,heart rate variability,Bagaimana cara memperoleh nilai indeks HRV?,Melalui rekaman EKG dengan panjang berapapun
2,heart rate variability,Apa perbedaan long-term HRV dan short-term HRV?,Perbedaannya terletak pada panjang rekaman detak jantung yang digunakan untuk perhitungan indeks HRV
3,long-term HRV,Berapa lama waktu rekaman EKG yang umumnya digunakan untuk menghitung long-term HRV?,24 jam
4,long-term HRV,Apakah ada variasi perhitungan long-term HRV selain rekaman 24 jam?,"Ya, ada"
...,...,...,...
213,confusion matrix,Apa nilai False Negative (FN) dalam confusion matrix?,Nilai False Negative (FN) dalam confusion matrix adalah jumlah prediksi negatif yang salah.
214,confusion matrix,Apa nilai True Positive (TP) dalam confusion matrix?,Nilai True Positive (TP) dalam confusion matrix adalah jumlah prediksi positif yang benar.
215,confusion matrix,Apa rumus untuk menghitung akurasi?,Rumus untuk menghitung akurasi adalah: Akurasi = (TP + TN) / (TP + TN + FP + FN)
216,AUC-ROC,Apa rumus untuk menghitung TPR dan FPR?,TPR = Sensitivitas\nFPR = 1 - Sensitivitas


### Generate Q/A Pairs for Bab 1

In [ ]:
df_qa = generate_qa(filepath="/content/Skripsi-Bab 1.pdf")
display(df_qa.sample(20))
df_qa.to_excel("df_qa_bab_1_qa-10.xlsx")
df_qa.to_csv("df_qa_bab_1_qa-10.csv")

CHUNK PAGE    : 1
DOCUMENT PAGE : 1
TIME TAKEN    : 0:00:12.378439
CHUNK PAGE    : 2
DOCUMENT PAGE : 1
TIME TAKEN    : 0:00:08.007702
CHUNK PAGE    : 3
DOCUMENT PAGE : 2
TIME TAKEN    : 0:00:12.576519
CHUNK PAGE    : 4
DOCUMENT PAGE : 2
 Error 'JSONDecodeError' at chunk 4, document page 2
 Skipping...
TIME TAKEN    : 0:00:23.042583
CHUNK PAGE    : 5
DOCUMENT PAGE : 2
TIME TAKEN    : 0:00:03.402442
CHUNK PAGE    : 6
DOCUMENT PAGE : 3
TIME TAKEN    : 0:00:10.306716
CHUNK PAGE    : 7
DOCUMENT PAGE : 3
TIME TAKEN    : 0:00:12.572916
CHUNK PAGE    : 8
DOCUMENT PAGE : 4
TIME TAKEN    : 0:00:10.787520
CHUNK PAGE    : 9
DOCUMENT PAGE : 4
TIME TAKEN    : 0:00:09.725631
CHUNK PAGE    : 10
DOCUMENT PAGE : 5


TIME TAKEN    : 0:00:16.678971
CHUNK PAGE    : 11
DOCUMENT PAGE : 5
TIME TAKEN    : 0:00:10.531068
CHUNK PAGE    : 12
DOCUMENT PAGE : 5
TIME TAKEN    : 0:00:06.714085
CHUNK PAGE    : 13
DOCUMENT PAGE : 6
 Error 'JSONDecodeError' at chunk 13, document page 6
 Skipping...
TIME TAKEN    : 0:00:08.901883
CHUNK PAGE    : 14
DOCUMENT PAGE : 6
TIME TAKEN    : 0:00:10.263407
CHUNK PAGE    : 15
DOCUMENT PAGE : 6
TIME TAKEN    : 0:00:16.706381
CHUNK PAGE    : 16
DOCUMENT PAGE : 7
TIME TAKEN    : 0:00:13.425666
CHUNK PAGE    : 17
DOCUMENT PAGE : 7
TIME TAKEN    : 0:00:10.787669
CHUNK PAGE    : 18
DOCUMENT PAGE : 7
 Error 'JSONDecodeError' at chunk 18, document page 7
 Skipping...
TIME TAKEN    : 0:00:14.480957
CHUNK PAGE    : 19
DOCUMENT PAGE : 8
TIME TAKEN    : 0:00:12.462704
CHUNK PAGE    : 20
DOCUMENT PAGE : 8
TIME TAKEN    : 0:00:13.531109
CHUNK PAGE    : 21
DOCUMENT PAGE : 8
TIME TAKEN    : 0:00:06.110584
CHUNK PAGE    : 22
DOCUMENT PAGE : 9


TIME TAKEN    : 0:00:24.976623
CHUNK PAGE    : 23
DOCUMENT PAGE : 9
TIME TAKEN    : 0:00:08.247873
Time taken       : 0:04:59.927688
Num. of Chunks   : 23


,topic,Q,A
137,penggunaan rekaman HRV,"Selain untuk penelitian, apa kegunaan lain dari rekaman HRV 5 menit?",Rekaman HRV 5 menit juga dapat digunakan sebagai alat diagnosis dini dan monitor kesehatan bagi masyarakat umum.
53,Gelombang EKG,Apa yang direpresentasikan oleh gelombang T pada EKG?,Gelombang T pada EKG merepresentasikan aktivitas repolarisasi ventrikel kanan dan kiri.
24,ECG (Electrocardiogram),Apa yang dimaksud dengan EKG?,EKG adalah rekaman aktivitas elektrik jantung yang ditangkap melalui bagian permukaan tubuh.
43,gelombang EKG,Apa yang terjadi pada ventrikel jantung setelah memompa darah?,Ventrikel jantung mengalami relaksasi
51,Gelombang EKG,Apa yang direpresentasikan oleh gelombang P pada EKG?,Gelombang P pada EKG merepresentasikan aktivitas atrium kanan dan kiri.
5,penyakit jantung,Berapa perkiraan jumlah kematian akibat CVD pada tahun 2021?,"17,9 juta"
101,klasifikasi resiko CHF menggunakan HRV,Apa yang dimaksud dengan CHF?,Congestive Heart Failure
47,Heart Rate Variability (HRV),Bagaimana cara memperoleh interval antara dua detak jantung?,Interval antara dua detak jantung dapat diperoleh dengan menghitung lama waktu interval antara dua gelombang PQRST yang berturut-turut.
31,Elektrokardiogram (EKG),Apa yang terjadi pada atrium jantung saat gelombang P muncul pada EKG?,Menerima darah dari seluruh tubuh
97,klasifikasi resiko CHF menggunakan HRV,Apa tahap kedua dalam model klasifikasi multi-tahap Chen dkk?,Klasifikasi tingkat keparahan CHF (NYHA kelas I-II & NYHA kelas III-IV)


### Generate Q/A Pairs for Bab 3

In [ ]:
df_qa = generate_qa(filepath="/content/Skripsi-Bab 3.pdf")
display(df_qa.sample(20))
df_qa.to_excel("df_qa_bab_3_qa-10.xlsx")
df_qa.to_csv("df_qa_bab_3_qa-10.csv")

CHUNK PAGE    : 1
DOCUMENT PAGE : 1
TIME TAKEN    : 0:00:11.080925
CHUNK PAGE    : 2
DOCUMENT PAGE : 1
TIME TAKEN    : 0:00:08.938043
CHUNK PAGE    : 3
DOCUMENT PAGE : 2
TIME TAKEN    : 0:00:19.236079
CHUNK PAGE    : 4
DOCUMENT PAGE : 2
TIME TAKEN    : 0:00:13.340007
CHUNK PAGE    : 5
DOCUMENT PAGE : 3
TIME TAKEN    : 0:00:10.033479
CHUNK PAGE    : 6
DOCUMENT PAGE : 3
TIME TAKEN    : 0:00:04.074418
CHUNK PAGE    : 7
DOCUMENT PAGE : 4
TIME TAKEN    : 0:00:15.516576
CHUNK PAGE    : 8
DOCUMENT PAGE : 5
TIME TAKEN    : 0:00:12.329055
CHUNK PAGE    : 9
DOCUMENT PAGE : 5
TIME TAKEN    : 0:00:13.069697
CHUNK PAGE    : 10
DOCUMENT PAGE : 6
TIME TAKEN    : 0:00:06.562895
CHUNK PAGE    : 11
DOCUMENT PAGE : 7
TIME TAKEN    : 0:00:29.724374
CHUNK PAGE    : 12
DOCUMENT PAGE : 7


TIME TAKEN    : 0:00:22.105651
CHUNK PAGE    : 13
DOCUMENT PAGE : 8
TIME TAKEN    : 0:00:17.495036
CHUNK PAGE    : 14
DOCUMENT PAGE : 8
TIME TAKEN    : 0:00:05.059840
CHUNK PAGE    : 15
DOCUMENT PAGE : 9
TIME TAKEN    : 0:00:19.255909
CHUNK PAGE    : 16
DOCUMENT PAGE : 9
TIME TAKEN    : 0:00:10.348186
CHUNK PAGE    : 17
DOCUMENT PAGE : 10
TIME TAKEN    : 0:00:10.189061
CHUNK PAGE    : 18
DOCUMENT PAGE : 10
 Error 'JSONDecodeError' at chunk 18, document page 10
 Skipping...
TIME TAKEN    : 0:00:04.363083
CHUNK PAGE    : 19
DOCUMENT PAGE : 11
TIME TAKEN    : 0:00:05.986217
CHUNK PAGE    : 20
DOCUMENT PAGE : 11
TIME TAKEN    : 0:00:09.966135
CHUNK PAGE    : 21
DOCUMENT PAGE : 12
TIME TAKEN    : 0:00:09.666324
CHUNK PAGE    : 22
DOCUMENT PAGE : 12
TIME TAKEN    : 0:00:12.820240
CHUNK PAGE    : 23
DOCUMENT PAGE : 13
TIME TAKEN    : 0:00:11.576767
CHUNK PAGE    : 24
DOCUMENT PAGE : 13
TIME TAKEN    : 0:00:03.918021
CHUNK PAGE    : 25
DOCUMENT PAGE : 14
TIME TAKEN    : 0:00:11.255975
CHUNK PA

TIME TAKEN    : 0:00:17.167493
CHUNK PAGE    : 36
DOCUMENT PAGE : 20


TIME TAKEN    : 0:00:21.060878
CHUNK PAGE    : 37
DOCUMENT PAGE : 20
TIME TAKEN    : 0:00:03.925456
CHUNK PAGE    : 38
DOCUMENT PAGE : 21
 Error 'JSONDecodeError' at chunk 38, document page 21
 Skipping...
TIME TAKEN    : 0:00:22.417739
CHUNK PAGE    : 39
DOCUMENT PAGE : 21
TIME TAKEN    : 0:00:14.698818
CHUNK PAGE    : 40
DOCUMENT PAGE : 22
TIME TAKEN    : 0:00:12.827978
CHUNK PAGE    : 41
DOCUMENT PAGE : 23
TIME TAKEN    : 0:00:12.371640
CHUNK PAGE    : 42
DOCUMENT PAGE : 23
TIME TAKEN    : 0:00:13.705553
CHUNK PAGE    : 43
DOCUMENT PAGE : 24
TIME TAKEN    : 0:00:11.021471
CHUNK PAGE    : 44
DOCUMENT PAGE : 24
TIME TAKEN    : 0:00:08.709785
CHUNK PAGE    : 45
DOCUMENT PAGE : 25
TIME TAKEN    : 0:00:17.703046
CHUNK PAGE    : 46
DOCUMENT PAGE : 25
TIME TAKEN    : 0:00:03.074386
Time taken       : 0:10:06.234971
Num. of Chunks   : 46


,topic,Q,A
278,xgboost_filename,Apa saja observasi yang termasuk dalam himpunan 𝐼3?,Observasi ke-1 dan ke-3.
291,inference process,Bagaimana pemetaan seluruh observasi pada data dummy ke suatu daun?,𝑓1(𝒙1)=𝑤𝑞(𝒙1)=𝑤3=−1\n𝑓1(𝒙2)=𝑤𝑞(𝒙2)=𝑤1=1\n𝑓1(𝒙3)=𝑤𝑞(𝒙3)=𝑤3=−1\n𝑓1(𝒙4)=𝑤𝑞(𝒙4)=𝑤1=1\n𝑓1(𝒙5)=𝑤𝑞(𝒙5)=𝑤2=2
240,Pohon Keputusan,Apa itu NNI_20?,Teks tidak memberikan informasi tentang NNI_20.
185,Classification and Regression Tree (CART),Apa saja aplikasi dari algoritma CART?,"Aplikasi dari algoritma CART antara lain: klasifikasi pelanggan, prediksi churn, deteksi penipuan, dan analisis sentimen."
30,Gradient Boosting Machines,Jelaskan fungsi dari bobot daun CART ke -𝑡 terhadap vektor input 𝒙𝑖 dan nilai prediksi model pada iterasi ke -(𝑡−1).,"𝑓𝑡(𝑥𝑖) merupakan nilai output berupa bobot daun CART ke -𝑡 terhadap vektor input 𝒙𝑖, sedangkan 𝑦̂𝑖(𝑡−1) adalah nilai prediksi model pada iterasi ke -(𝑡−1)."
127,XGBoost,Sebutkan langkah-langkah untuk membentuk CART dalam XGBoost,1. Buat simpul akar yang merepresentasikan seluruh data\n2. Bagi data ke dalam dua simpul anak dengan menggunakan atribut yang memiliki nilai gain terbesar\n3. Ulangi langkah 2 sampai tingkat batas kedalaman pohon tercapai
290,inference process,Apa yang dimaksud dengan q(xi)?,q(𝒙𝑖) memetakan vektor input 𝒙𝑖 ke indeks daun yang sesuai dengan berbagai keputusan yang dilalui dari simpul akar hingga mencapai daun.
176,Pohon Keputusan Regresi,Apa tujuan dari pembagian data pada pohon keputusan?,"Untuk memisahkan data menjadi kelompok-kelompok yang lebih homogen, sehingga dapat meningkatkan akurasi prediksi."
223,penentuan parameter pohon keputusan,"Bagaimana cara menentukan pasangan (fitur, threshold) untuk menjadi simpul?","Dipilih pasangan (fitur, threshold) pertama yang dikunjungi."
122,similarity score,Apa fungsi dari percabangan pada pohon?,Percabangan berfungsi untuk memisahkan data berdasarkan kondisi tertentu.


### Generate Q/A Pairs for Bab 4

In [ ]:
df_qa = generate_qa(filepath="/content/Skripsi-Bab 4.pdf")
display(df_qa.sample(20))
df_qa.to_excel("df_qa_bab_4_qa-10.xlsx")
df_qa.to_csv("df_qa_bab_4_qa-10.csv")

CHUNK PAGE    : 1
DOCUMENT PAGE : 1
TIME TAKEN    : 0:00:10.565329
CHUNK PAGE    : 2
DOCUMENT PAGE : 2
TIME TAKEN    : 0:00:13.187601
CHUNK PAGE    : 3
DOCUMENT PAGE : 3
TIME TAKEN    : 0:00:08.066833
CHUNK PAGE    : 4
DOCUMENT PAGE : 4
TIME TAKEN    : 0:00:11.236694
CHUNK PAGE    : 5
DOCUMENT PAGE : 5
TIME TAKEN    : 0:00:10.976914
CHUNK PAGE    : 6
DOCUMENT PAGE : 5
TIME TAKEN    : 0:00:15.459571
CHUNK PAGE    : 7
DOCUMENT PAGE : 5


TIME TAKEN    : 0:00:24.045794
CHUNK PAGE    : 8
DOCUMENT PAGE : 6
 Error 'JSONDecodeError' at chunk 8, document page 6
 Skipping...
TIME TAKEN    : 0:00:06.348261
CHUNK PAGE    : 9
DOCUMENT PAGE : 6
TIME TAKEN    : 0:00:06.755365
CHUNK PAGE    : 10
DOCUMENT PAGE : 7
TIME TAKEN    : 0:00:05.965138
CHUNK PAGE    : 11
DOCUMENT PAGE : 7
TIME TAKEN    : 0:00:05.066592
CHUNK PAGE    : 12
DOCUMENT PAGE : 8
TIME TAKEN    : 0:00:11.690701
CHUNK PAGE    : 13
DOCUMENT PAGE : 8
TIME TAKEN    : 0:00:11.270346
CHUNK PAGE    : 14
DOCUMENT PAGE : 9


 Error 'JSONDecodeError' at chunk 14, document page 9
 Skipping...
TIME TAKEN    : 0:00:17.599647
CHUNK PAGE    : 15
DOCUMENT PAGE : 9
TIME TAKEN    : 0:00:12.228513
CHUNK PAGE    : 16
DOCUMENT PAGE : 9
 Error 'JSONDecodeError' at chunk 16, document page 9
 Skipping...
TIME TAKEN    : 0:00:09.152168
CHUNK PAGE    : 17
DOCUMENT PAGE : 9
TIME TAKEN    : 0:00:04.592728
CHUNK PAGE    : 18
DOCUMENT PAGE : 10
TIME TAKEN    : 0:00:10.879969
CHUNK PAGE    : 19
DOCUMENT PAGE : 11
TIME TAKEN    : 0:00:08.613183
CHUNK PAGE    : 20
DOCUMENT PAGE : 12
TIME TAKEN    : 0:00:16.838764
CHUNK PAGE    : 21
DOCUMENT PAGE : 12
TIME TAKEN    : 0:00:10.695385
CHUNK PAGE    : 22
DOCUMENT PAGE : 13
TIME TAKEN    : 0:00:14.116027
CHUNK PAGE    : 23
DOCUMENT PAGE : 13
TIME TAKEN    : 0:00:15.368801
CHUNK PAGE    : 24
DOCUMENT PAGE : 14
TIME TAKEN    : 0:00:12.798924
CHUNK PAGE    : 25
DOCUMENT PAGE : 14
TIME TAKEN    : 0:00:12.098924
CHUNK PAGE    : 26
DOCUMENT PAGE : 15
TIME TAKEN    : 0:00:15.349885
CHUNK PAGE

,topic,Q,A
273,XGBoost,Apa kepanjangan dari XGBoost?,eXtreme Gradient Boosting
245,XGBoost Spesifisitas,"Nilai akurasi, sensitivitas, spesifisitas, dan skor AUC yang dicapai model XGBoost yang dilatih untuk memaksimalkan nilai spesifisitas pada data train set adalah berapa?","Akurasi: 1,0\nSensitivitas: 1,0\nSpesifisitas: 1,0\nSkor AUC: 1,0"
125,Dataset,Apa nama database yang memiliki jumlah rekaman data terbanyak?,NSR2DB
142,Data yang digunakan,Berapa jumlah keseluruhan segmentasi yang digunakan dalam penelitian ini?,4567 segmentasi
68,Penentuan Lokasi R-peak Ektopik pada EKG,Bagaimana cara mengevaluasi akurasi interpolasi linear dalam menentukan lokasi R-peak ektopik?,"Akurasi interpolasi linear dapat dievaluasi dengan membandingkan nilai R-peak yang diperkirakan dengan nilai R-peak yang sebenarnya, menggunakan metrik seperti mean absolute error atau root mean square error."
32,Penghitungan Indeks HRV pada Pasien CHF,Apa yang direpresentasikan oleh array pada Gambar 4.4?,Array pada Gambar 4.4 merepresentasikan barisan titik lokasi waktu R-peak pertama pada suatu rekaman.
47,Pembersihan dan Praproses Data,Bagaimana cara menangani ectopic beat?,Dengan menandai dan menghapusnya dari data.
182,Grid Search,Jelaskan ilustrasi Grid Search pada Gambar 4.7,"Gambar 4.7 mengilustrasikan proses Grid Search, dimana terdapat beberapa hyperparameter yang dioptimisasi dengan berbagai pilihan nilai. Kombinasi dari nilai-nilai hyperparameter ini akan dievaluasi menggunakan teknik K-Fold Cross Validation untuk menentukan himpunan hyperparameter terbaik."
96,resampling ECG,"Pada gambar 4.5, segmen dengan ID nsrdb -16265 -300-600 memiliki nilai interval RR berapa setelah dilakukan resampling?",609.375
74,Pemrosesan Sinyal EKG,Apa itu segmen dengan nilai id: nsrdb -16265 -300-600?,Segmen dengan nilai id: nsrdb -16265 -300-600 adalah segmen data EKG dengan panjang 300-600 titik waktu dan memiliki id nsrdb -16265


### Generate Q/A Pairs for Bab 5

In [ ]:
df_qa = generate_qa(filepath="/content/Skripsi-Bab 5.pdf")
display(df_qa.sample(20))
df_qa.to_excel("df_qa_bab_5_qa-10.xlsx")
df_qa.to_csv("df_qa_bab_5_qa-10.csv")

CHUNK PAGE    : 1
DOCUMENT PAGE : 1
TIME TAKEN    : 0:00:10.153313
CHUNK PAGE    : 2
DOCUMENT PAGE : 1
TIME TAKEN    : 0:00:13.099092
CHUNK PAGE    : 3
DOCUMENT PAGE : 1
TIME TAKEN    : 0:00:30.132468
CHUNK PAGE    : 4
DOCUMENT PAGE : 2
TIME TAKEN    : 0:00:09.552995
CHUNK PAGE    : 5
DOCUMENT PAGE : 2
TIME TAKEN    : 0:00:12.527851
Time taken       : 0:01:20.550747
Num. of Chunks   : 5


,topic,Q,A
24,Tugas Akhir_Deteksi Gagal Jantung Kongestif,Nilai akurasi tertinggi yang dicapai oleh model yang dilatih adalah berapa?,"0,966"
6,Klasifikasi CHF Menggunakan HRV,Berapa jumlah data yang digunakan untuk kelas CHF?,44
13,Skenario Tuning,Berdasarkan metrik evaluasi apa saja hyperparameter tuning dilakukan?,"Akurasi, sensitivitas, spesifisitas, dan skor AUC"
19,Pentingnya Hyperparameter Tuning,Mengapa hyperparameter tuning penting dalam proses pelatihan model?,Hyperparameter tuning membantu menemukan kombinasi hyperparameter yang optimal untuk memaksimalkan kinerja model pada tugas tertentu.
25,Tugas Akhir_Deteksi Gagal Jantung Kongestif,Apa kepanjangan dari AUC?,Area Under the Curve
16,Sensitivitas,Jelaskan tentang metrik evaluasi sensitivitas.,Sensitivitas mengukur kemampuan model dalam mengidentifikasi pasien dengan CHF yang sebenarnya menderita CHF.
15,Hasil Tuning Akurasi,Berapa nilai akurasi yang dicapai model saat hyperparameter tuning berfokus pada metrik akurasi?,"0,954"
7,Klasifikasi CHF Menggunakan HRV,Berapa persentase data yang digunakan untuk train set?,70%
8,Klasifikasi CHF Menggunakan HRV,Berapa jumlah sampel dalam train set?,2.326
3,Klasifikasi CHF Menggunakan HRV,Apa yang dihitung dari lokasi R-peak?,Interval RR


### Generate Q/A Pairs for Abstrak & Judul

In [ ]:
df_qa = generate_qa(filepath="/content/Skripsi-Judul & Abstrak.pdf")
display(df_qa.sample(20))
df_qa.to_excel("df_qa_judul_abstrak_qa-10.xlsx")
df_qa.to_csv("df_qa_judul_abstrak_qa-10.csv")

CHUNK PAGE    : 1
DOCUMENT PAGE : 1
TIME TAKEN    : 0:00:08.865602
CHUNK PAGE    : 2
DOCUMENT PAGE : 2
TIME TAKEN    : 0:00:11.229581
CHUNK PAGE    : 3
DOCUMENT PAGE : 2
TIME TAKEN    : 0:00:12.908326
CHUNK PAGE    : 4
DOCUMENT PAGE : 2
TIME TAKEN    : 0:00:09.533625
CHUNK PAGE    : 5
DOCUMENT PAGE : 2
TIME TAKEN    : 0:00:10.223165
CHUNK PAGE    : 6
DOCUMENT PAGE : 3
TIME TAKEN    : 0:00:11.873173
CHUNK PAGE    : 7
DOCUMENT PAGE : 3
TIME TAKEN    : 0:00:10.580889
CHUNK PAGE    : 8
DOCUMENT PAGE : 3
TIME TAKEN    : 0:00:10.350260
CHUNK PAGE    : 9
DOCUMENT PAGE : 3
TIME TAKEN    : 0:00:06.302740
Time taken       : 0:01:41.003022
Num. of Chunks   : 9


,topic,Q,A
1,Congestive Heart Failure,Apa kepanjangan dari CHF?,Congestive Heart Failure
18,Analisis Kinerja Model XGBoost Dalam Mendeteksi Congestive Heart Failure Pada Short-term Heart Rate Variability,Apa kelebihan model XGBoost dalam mendeteksi CHF?,Memiliki akurasi dan efisiensi yang tinggi
27,model XGBoost,Untuk apa model XGBoost digunakan dalam penelitian ini?,"Dalam penelitian ini, model XGBoost digunakan untuk mendeteksi keberadaan CHF menggunakan HRV sebagai fitur bagi model."
58,Studi Analisis Performa Model XGBoost untuk Deteksi Congestive Heart Failure pada Variabilitas Denyut Jantung Jangka Pendek,Apa kepanjangan dari WHO?,World Health Organization
15,Analisis Kinerja Model XGBoost Dalam Mendeteksi Congestive Heart Failure Pada Short-term Heart Rate Variability,Apa itu heart rate variability?,Variasi waktu antara detak jantung
28,dataset,Dari mana dataset yang digunakan dalam penelitian ini berasal?,"Dataset yang digunakan dalam penelitian ini berasal dari empat database yang berbeda yang diambil dari situs PhysioNet, yaitu NSRDB dan NSR2DB sebagai kelas sehat dan CHFDB dan CHF2DB."
39,Metode,Apa metode yang digunakan untuk melatih model XGBoost?,Kombinasi teknik Grid Search dan K-Fold Cross Validation dengan nilai 𝐾=10
52,Studi Analisis Performa Model XGBoost untuk Deteksi Congestive Heart Failure pada Variabilitas Denyut Jantung Jangka Pendek,Apa program studi dari penulis tesis ini?,Matematika
75,Heart Rate Variability,"Berapa nilai akurasi, sensitivitas, spesifisitas, dan skor AUC terbaik yang dicapai oleh model XGBoost yang dioptimalkan untuk spesifisitas?","0,962; 0,931; 0,971; 0,951."
82,congestive heart failure,Apa saja kata kunci yang digunakan dalam penelitian ini?,"Congestive heart failure, Heart rate variability, Short-term HRV, XGBoost"


### Inspecting the DataFrame

In [ ]:
# for easier dataframe QnA read
pd.set_option('display.max_colwidth', None)

print("BAB 1")
display(pd.read_csv("df_qa_bab_1_qa-10.csv", index_col=[0]).sample(5))
print("\nBAB 2")
display(pd.read_csv("df_qa_bab_2_qa-10.csv", index_col=[0]).sample(5))
print("\nBAB 3")
display(pd.read_csv("df_qa_bab_3_qa-10.csv", index_col=[0]).sample(5))
print("\nBAB 4")
display(pd.read_csv("df_qa_bab_4_qa-10.csv", index_col=[0]).sample(5))
print("\nBAB 5")
display(pd.read_csv("df_qa_bab_5_qa-10.csv", index_col=[0]).sample(5))

BAB 1


,topic,Q,A
66,HRV dan Autonomic Nervous System,Apa yang dimaksud dengan Autonomic Nervous System (ANS)?,"ANS adalah sistem saraf yang mengontrol proses fisiologis yang tidak disadari, seperti detak jantung, tekanan darah, pernafasan, pencernaan, dan aktivitas seksual."
58,Pengukuran Interval RR,Apa singkatan dari R peak?,R peak adalah singkatan dari puncak gelombang R.
57,Pengukuran Interval RR,Jelaskan bagaimana cara mengukur interval RR.,"Interval RR diukur dengan cara menghitung lama waktu antara dua gelombang PQRST yang berturut-turut, dengan titik patokan pengukuran adalah puncak gelombang R."
7,penyakit jantung,Apa salah satu jenis penyakit jantung?,Congestive heart failure atau gagal jantung kongestif
129,XGBoost,Apa keuntungan menggunakan XGBoost?,Tidak disebutkan dalam teks yang diberikan.



BAB 2


,topic,Q,A
57,preprocessed RR interval,Apa yang dimaksud dengan MIN_HR?,MIN_HR adalah nilai minimum HR dari barisan HR yang diperoleh melalui persamaan (2.1).
43,Sinyal EKG,Apa yang dimaksud dengan sinyal EKG?,Sinyal EKG adalah rekaman aktivitas listrik jantung
62,Indeks frequency domain HRV,Jelaskan pengertian dari indeks LF/HF,Perbandingan kekuatan spektral pada range frekuensi LF dengan kekuatan spektral pada range frekuensi HF
125,machine learning,"Apa pengertian dari ""building block"" dalam ensemble learning?",Model-model simpel yang digabungkan untuk membentuk satu model
185,Gradient Boosting,Jelaskan langkah-langkah untuk membangun pohon regresi pada residu\n,"a. Fit suatu Regression Tree terhadap residu 𝑟𝑖𝑚 dan buat terminal region 𝑅𝑗𝑚 untuk setiap leaf 𝑗=1,2,…,𝐽𝑚"



BAB 3


,topic,Q,A
140,Batas Tingkat Kedalaman Pohon CART,Berapakah batas tingkat kedalaman pohon CART yang digunakan dalam simulasi XGBoost?,2.
325,Data Dummy,"Pada dokumen ""Pengaruh Fitur-Fitur Fisiologis Pada Prediksi Penyakit Jantung Menggunakan Algoritma XGBoost"", berapa jumlah observasi pada data dummy?",5
171,Pohon Keputusan Regresi,Apa itu Similarity KIR?,Nilai Similarity Score pada cabang kiri pohon keputusan.
267,Pohon Keputusan,Apa yang dimaksud dengan I₁?,I₁ adalah himpunan indeks observasi yang termasuk dalam daun pertama.
264,Algoritma C4.5,Apakah simpul yang dihapus memiliki selisih nilai Gain dengan γ lebih besar atau kurang dari 0?,Kurang dari 0



BAB 4


,topic,Q,A
225,XGBoost,Bagaimana performa model XGBoost pada data train set?,"Akurasi: 0,998,\nSensitivitas: 0,996,\nSpesifisitas: 0,998,\nAUC: 0,997"
192,Grid Search dan K-Fold Cross Validation,Apa itu Objective Function dalam Grid Search?,Metrik yang didefinisikan ketika menjalankan Grid Search untuk mengevaluasi performa model pada setiap pilihan himpunan hyperparameter.
184,Grid Search,Apa itu K-Fold Cross Validation?,"K-Fold Cross Validation adalah teknik validasi pemilihan model dengan cara membagi dataset menjadi 𝐾 subhimpunan, dimana 𝐾−1 subhimpunan digunakan untuk melatih model dan himpunan terakhir digunakan untuk mengevaluasi model yang telah dilatih."
153,filename: skripsi-julius-1301154081.pdf,Bagaimana teknik pembagian dataset yang digunakan?,Teknik pembagian dataset yang digunakan adalah train-test split dengan rasio 70% untuk train set dan 30% untuk test set.
86,resampling data,Apa saja aplikasi dari resampling data?,"Aplikasi dari resampling data antara lain validasi silang, pemilihan model, dan estimasi interval kepercayaan."



BAB 5


,topic,Q,A
6,Klasifikasi CHF Menggunakan HRV,Berapa jumlah data yang digunakan untuk kelas CHF?,44
33,model prediktif gagal jantung kongestif,Apa saran penelitian yang diberikan oleh peneliti?,Mengimplementasikan metode reduksi dimensi fitur untuk mengatasi permasalahan multikolinearitas
31,model prediktif gagal jantung kongestif,Nilai AUC terbaik yang dicapai pada optimalisasi terhadap metrik sensitivitas adalah berapa?,"0,948"
8,Klasifikasi CHF Menggunakan HRV,Berapa jumlah sampel dalam train set?,2.326
0,Klasifikasi CHF Menggunakan HRV,Apa yang diekstrak dari sinyal EKG untuk melakukan klasifikasi CHF?,Data short-term HRV


## Prepare Dataset

In [ ]:
%%capture
!pip install -U bitsandbytes
!pip install transformers==4.36.2
!pip install -U peft
!pip install -U accelerate
!pip install -U trl
!pip install datasets==2.16.0
!pip install sentencepiece

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig,\
                        HfArgumentParser,TrainingArguments,pipeline, logging
from peft import LoraConfig, PeftModel, prepare_model_for_kbit_training, get_peft_model
import os,torch
from datasets import load_dataset
from trl import SFTTrainer
import pandas as pd
import pyarrow as pa
import pyarrow.dataset as ds
import pandas as pd
from datasets import Dataset
import re

In [ ]:
secret_hf = os.environ["HUGGINGFACEHUB_API_TOKEN"]
!huggingface-cli login --token $secret_hf

Token will not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [ ]:
df_qa_list = [
    "/content/df_qa_bab_1_qa-10.csv",
    "/content/df_qa_bab_2_qa-10.csv",
    "/content/df_qa_bab_3_qa-10.csv",
    "/content/df_qa_bab_4_qa-10.csv",
    "/content/df_qa_bab_5_qa-10.csv",
    "/content/df_qa_judul_abstrak_qa-10.csv"
]

df = pd.DataFrame()
for df_qa_file in df_qa_list:
    _df = pd.read_csv(df_qa_file, index_col=[0])
    _df["bab"] = df_qa_file.split("/")[-1][:-12]
    df = pd.concat([df, _df], axis=0)

# build training dataset with the right format
df['text_mistral_instruct'] = '<s>[INST]@ChatBotSkripsi. ' + df['Q'] +'[/INST]'+ df['A'] + '</s>'
df = df.reset_index(drop=True)
display(df.sample(8))

,topic,Q,A,bab,text_mistral_instruct
928,XGBoost,Jelaskan tentang hyperparameter learning_rate dalam XGBoost!,"Hyperparameter learning_rate dalam XGBoost mengontrol seberapa besar bobot pembaruan pada setiap iterasi model.\nNilai yang lebih tinggi menghasilkan pembaruan yang lebih besar, sementara nilai yang lebih rendah menghasilkan pembaruan yang lebih kecil.",df_qa_bab,"<s>[INST]@ChatBotSkripsi. Jelaskan tentang hyperparameter learning_rate dalam XGBoost![/INST]Hyperparameter learning_rate dalam XGBoost mengontrol seberapa besar bobot pembaruan pada setiap iterasi model.\nNilai yang lebih tinggi menghasilkan pembaruan yang lebih besar, sementara nilai yang lebih rendah menghasilkan pembaruan yang lebih kecil.</s>"
182,Heart Rate Variability (HRV),Apa yang diukur oleh indeks frequency domain HRV?,Indeks frequency domain HRV mengukur jumlah energi atau power dalam component band.,df_qa_bab,<s>[INST]@ChatBotSkripsi. Apa yang diukur oleh indeks frequency domain HRV?[/INST]Indeks frequency domain HRV mengukur jumlah energi atau power dalam component band.</s>
1070,congestive heart failure,Apa itu congestive heart failure (CHF)?,"CHF adalah kondisi dimana jantung tidak dapat memompa darah yang cukup ke seluruh tubuh, sehingga menyebabkan beberapa bagian tubuh tidak mendapatkan suplai darah yang cukup.",df_qa_judul_abstr,"<s>[INST]@ChatBotSkripsi. Apa itu congestive heart failure (CHF)?[/INST]CHF adalah kondisi dimana jantung tidak dapat memompa darah yang cukup ke seluruh tubuh, sehingga menyebabkan beberapa bagian tubuh tidak mendapatkan suplai darah yang cukup.</s>"
1019,Tugas Akhir_Deteksi Gagal Jantung Kongestif,"Saat proses hyperparameter tuning berfokus pada optimisasi sensitivitas, nilai metrik apa saja yang dicapai oleh model yang dilatih?","Akurasi: 0,966\nSensitivitas: 0,977\nSpesifisitas: 0,963\nAUC: 0,970",df_qa_bab,"<s>[INST]@ChatBotSkripsi. Saat proses hyperparameter tuning berfokus pada optimisasi sensitivitas, nilai metrik apa saja yang dicapai oleh model yang dilatih?[/INST]Akurasi: 0,966\nSensitivitas: 0,977\nSpesifisitas: 0,963\nAUC: 0,970</s>"
169,short-term HRV,Apakah short-term HRV dapat memberikan informasi yang berbeda dari long-term HRV?,Tidak disebutkan dalam teks yang diberikan,df_qa_bab,<s>[INST]@ChatBotSkripsi. Apakah short-term HRV dapat memberikan informasi yang berbeda dari long-term HRV?[/INST]Tidak disebutkan dalam teks yang diberikan</s>
649,XGBoost,Apa kepanjangan dari XGBoost?,eXtreme Gradient Boosting,df_qa_bab,<s>[INST]@ChatBotSkripsi. Apa kepanjangan dari XGBoost?[/INST]eXtreme Gradient Boosting</s>
851,Data yang digunakan,Berapa jumlah segmentasi yang digunakan dari database NSRDB setelah dilakukan filter terhadap % ectopic beat?,780 segmentasi,df_qa_bab,<s>[INST]@ChatBotSkripsi. Berapa jumlah segmentasi yang digunakan dari database NSRDB setelah dilakukan filter terhadap % ectopic beat?[/INST]780 segmentasi</s>
752,Pembersihan dan Praproses Data,Apa itu ectopic beat?,"Detak jantung yang berasal dari luar nodus sinoatrial, biasanya berupa detak jantung yang prematur.",df_qa_bab,"<s>[INST]@ChatBotSkripsi. Apa itu ectopic beat?[/INST]Detak jantung yang berasal dari luar nodus sinoatrial, biasanya berupa detak jantung yang prematur.</s>"


In [ ]:
df.to_excel("df_qa_skripsi.xlsx")
df.to_csv("df_qa_skripsi.csv")

In [ ]:
# df = pd.read_csv("df_qa_skripsi.csv", index_col=[0])

# convert to dataset object
# dataset = ds.dataset(pa.Table.from_pandas(df).to_batches())
dataset = Dataset(pa.Table.from_pandas(df.drop(["filename", "topic", "Q", "A", "bab"], axis=1)))

In [ ]:
dataset

Dataset({
    features: ['text'],
    num_rows: 716
})

## Prepare Model & Training

In [ ]:
base_model = "mistralai/Mistral-7B-Instruct-v0.2"
new_model = "rinogrego/Mistral-7B-Skripsi"

In [ ]:
# Load base model
bnb_config = BitsAndBytesConfig(
    load_in_4bit= True,
    bnb_4bit_quant_type= "nf4",
    bnb_4bit_compute_dtype= torch.bfloat16,
    bnb_4bit_use_double_quant= False,
)
model = AutoModelForCausalLM.from_pretrained(
        base_model,
        load_in_4bit=True,
        quantization_config=bnb_config,
        torch_dtype=torch.bfloat16,
        device_map="auto",
        trust_remote_code=True,
)

model.config.use_cache = False
model.config.pretraining_tp = 1
model.gradient_checkpointing_enable()

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(base_model, trust_remote_code=True)
tokenizer.padding_side = 'right'
tokenizer.pad_token = tokenizer.eos_token
tokenizer.add_eos_token = True
tokenizer.bos_token, tokenizer.eos_token

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/596 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/1.46k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/72.0 [00:00<?, ?B/s]

('<s>', '</s>')

In [ ]:
model = prepare_model_for_kbit_training(model)
peft_config = LoraConfig(
    lora_alpha=16,
    lora_dropout=0.1,
    r=64,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj","gate_proj"]
)
model = get_peft_model(model, peft_config)

In [ ]:
#Hyperparamter
training_arguments = TrainingArguments(
    output_dir="./results",
    num_train_epochs=1,
    per_device_train_batch_size=4,
    gradient_accumulation_steps=1,
    optim="paged_adamw_32bit",
    save_steps=50,
    logging_steps=10,
    learning_rate=2e-4,
    weight_decay=0.001,
    fp16=False,
    bf16=False,
    max_grad_norm=0.3,
    max_steps=-1,
    warmup_ratio=0.03,
    group_by_length=True,
    lr_scheduler_type="constant",
)

In [ ]:
# Setting sft parameters
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    peft_config=peft_config,
    max_seq_length= None,
    dataset_text_field="text",
    tokenizer=tokenizer,
    args=training_arguments,
    packing= False,
)

Map:   0%|          | 0/716 [00:00<?, ? examples/s]

## Start Training

In [ ]:
trainer.train()

Step,Training Loss
10,0.626800
20,0.677600
30,0.768500
40,1.025400
50,1.061400
60,0.893300
70,0.839800
80,0.870300
90,0.799700
100,0.778000


/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.

TrainOutput(global_step=179, training_loss=0.8096767803810162, metrics={'train_runtime': 1292.9202, 'train_samples_per_second': 0.554, 'train_steps_per_second': 0.138, 'total_flos': 3417302755442688.0, 'train_loss': 0.8096767803810162, 'epoch': 1.0})

## Save Model

In [ ]:
# saving adapter
trainer.model.save_pretrained(new_model)
model.config.use_cache = True
model.eval()

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): MistralForCausalLM(
      (model): MistralModel(
        (embed_tokens): Embedding(32000, 4096)
        (layers): ModuleList(
          (0-31): 32 x MistralDecoderLayer(
            (self_attn): MistralAttention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=4096, out_features=4096, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.1, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=64, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=64, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
              )
              (k_proj): lora.Linear4bit(
                (base_layer): Line

In [ ]:
# push
trainer.model.push_to_hub(new_model)

adapter_model.safetensors:   0%|          | 0.00/369M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/rinogrego/Mistral-7B-Skripsi/commit/0cd22e880d9b72e9e006b3e0e95e4bf1213ae274', commit_message='Upload model', commit_description='', oid='0cd22e880d9b72e9e006b3e0e95e4bf1213ae274', pr_url=None, pr_revision=None, pr_num=None)

## Inference the Model

In [ ]:
logging.set_verbosity(logging.CRITICAL)
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=500)

In [ ]:
# create helper function
def build_prompt(question):
  prompt=f"<s>[INST]@ChatBotSkripsi. {question} [/INST]"
  return prompt

In [ ]:
question = "Apa saja 2 kelas paling parah dari CHF menurut pembagian kelas fungsional NYHA?"
prompt = build_prompt(question)
result = pipe(prompt)
result[0]['generated_text']

<s>[INST]@ChatBotSkripsi. Apa saja 2 kelas paling parah dari CHF menurut pembagian kelas fungsional NYHA? [/INST]Kelas fungsional NYHA membagi CHF menjadi 4 kelas, yaitu NYHA Class I, NYHA Class II, NYHA Class III, dan NYHA Class IV. Kelas fungsional NYHA Class IV adalah kelas paling parah dari CHF. Kelas fungsional NYHA Class III juga termasuk kelas parah, namun lebih sedikit baik dari kelas fungsional NYHA Class IV. Kelas fungsional NYHA Class I dan NYHA Class II termasuk kelas lebih baik dari CHF. Kelas fungsional NYHA Class I adalah kelas baik, sedangkan kelas fungsional NYHA Class II adalah kelas sedang. Kelas fungsional NYHA Class III adalah kelas parah, sedangkan kelas fungsional NYHA Class IV adalah kelas parah yang lebih buruk.


In [ ]:
question = "Bagaimana rumus fungsi objective XGBoost?"
prompt = build_prompt(question)
result = pipe(prompt)

# print(result[0]['generated_text'])
result[0]['generated_text']

'<s>[INST]@ChatBotSkripsi. Bagaimana rumus fungsi objective XGBoost? [/INST]Rumus fungsi objective XGBoost adalah sumbu yang menjadi nilai target.\n\n𝑓𝑁(𝑓𝑁−1) =∑(∑(∑(∑(∑(∑(∑(∑(∑(∑(∑(∑(∑(∑(∑(∑(∑(∑(∑(∑(∑(∑(∑(∑(∑(∑(∑(∑(∑(∑(∑(∑(∑(∑(∑(∑(∑(∑(∑(∑(∑(∑(∑(∑(∑(∑(∑(∑(∑(∑(∑(∑(∑(∑(∑(∑(∑(∑(∑(∑(∑(∑(∑(∑(∑(∑(∑(∑(∑(∑(∑(∑(∑(∑(∑(∑(∑(∑(∑(∑(∑(∑(∑(∑(∑(∑(∑(∑(∑(∑(∑(∑(∑(∑(∑(∑(∑(∑(∑(∑(∑(∑(∑(∑(∑(∑(∑(∑(∑(∑(∑(∑(∑(∑(∑(∑(∑(∑(∑(∑(∑(∑(∑(∑(∑(∑(∑(∑(∑(∑(∑(∑(∑(∑(∑(∑(∑(∑(∑(∑(∑(∑(∑(∑(∑(∑(∑(∑(∑(∑(∑(∑(∑(∑(∑(∑(∑(∑(∑(∑(∑(∑(∑(∑(∑(∑(∑(∑(∑(∑(∑(∑(∑(∑(∑(∑(∑(∑(∑(∑(∑(∑(∑(∑(∑(∑(∑(∑(∑(∑(∑(∑(∑(∑(∑(∑(∑(∑(∑(∑(∑(∑(∑(∑(∑(∑(∑(∑(∑(∑(∑(∑'

## Merge base model with adapter (On Kaggle, seems GPU/CUDA memory issue...)

Go to the following kaggle notebook link for better training and to see merging process:
https://www.kaggle.com/code/rinogrego/fine-tuning-mistral-7b

In [ ]:
base_model = "mistralai/Mistral-7B-Instruct-v0.2"
new_model = "rinogrego/Mistral-7B-Skripsi"

In [ ]:
base_model_reload = AutoModelForCausalLM.from_pretrained(
    base_model,
    torch_dtype=torch.bfloat16,
    return_dict=True,
    low_cpu_mem_usage=True,
    device_map={'': 0},
    trust_remote_code=True,
    # offload_folder="offload/"
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/596 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

In [ ]:
# merge adopter with base model
model = PeftModel.from_pretrained(
    base_model_reload,
    new_model,
    device_map={'': 0},
    offload_folder="offload"
)
model = model.merge_and_unload()

# Reload tokenizer
tokenizer = AutoTokenizer.from_pretrained(base_model, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

RuntimeError: CUDA error: CUBLAS_STATUS_ALLOC_FAILED when calling `cublasCreate(handle)`

In [ ]:
# Error ref
# NotImplementedError: Cannot copy out of meta tensor; no data! #368
# - https://github.com/tloen/alpaca-lora/issues/368
# ValueError: We need an offload_dir to dispatch this model according to this device_map > ```python #86
# - https://github.com/OptimalScale/LMFlow/issues/86

In [ ]:
#push the model to hub
hf_name=new_model+'-Merged'
model.push_to_hub(hf_name)
tokenizer.push_to_hub(hf_name)

# References

- [Developing QA (question-answering applications) using OpenAI, Pinecone, and LangChain (OPL stack)](https://medium.com/@atef.ataya/developing-qa-question-answering-applications-using-openai-pinecone-and-langchain-opl-stack-90f0c9657588)
- https://github.com/lxe/simple-llm-finetuner/issues/38#issuecomment-1510880293
- https://github.com/lxe/simple-llm-finetuner/issues/38#issuecomment-1582466448
- [Fine Tune Large Language Model (LLM) on a Custom Dataset with QLoRA](https://dassum.medium.com/fine-tune-large-language-model-llm-on-a-custom-dataset-with-qlora-fb60abdeba07)
- [My experience on starting with fine tuning LLMs with custom data](https://www.reddit.com/r/LocalLLaMA/comments/14vnfh2/my_experience_on_starting_with_fine_tuning_llms/)
- [Train question answering model with custom dataset](https://datascience.stackexchange.com/questions/118417/train-question-answering-model-with-custom-dataset?rq=1)
- [Fine-tuning a pre-trained LLM for question-answering](https://stackoverflow.com/questions/76373220/fine-tuning-a-pre-trained-llm-for-question-answering)

In [ ]:
SYSTEM_PROMPT_FOR_QA = """# Answer the question given the following contexts.
# Give the metadata regarding the context like title, header, and page number.
# If there is no appropriate answer from the retrieved documents then say "The answer you are seeking for is not found from the documents".
# Example output
[
  {
    "filename": "sugar-report.pdf",
    "Q": "Berapa kadar gula yang telah saya konsumsi selama hari ini? Bagaimana analisisnya?",
    "A": "Berdasarkan laporan konsumsi gula anda selama satu minggu ini, anda telah mengonsumsi gula sebanyak 153 gram. Angka ini jauh melebihi batas yang telah disarankan oleh Kementerian Kesehatan Republik Indonesia.",
    "page_number": 30
  },
  {
    "filename": "report-34.pdf",
    "Q": "",
    "A": "",
    "page_number": 34
  }
]

NOTE:::: CHUNK PER PAGE/TWO-PAGES IF POSSIBLE

"""